<a href="https://colab.research.google.com/github/asad-777/Piaic_Gen-Ai/blob/main/tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install langchain langchain_google_genai -q -U

In [78]:
from google.colab import userdata
api_key=userdata.get("GEMINI_API_KEY")

In [79]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [80]:
llm = ChatGoogleGenerativeAI(
    api_key = api_key,
    model = "gemini-1.5-flash"
)

In [81]:
from langchain_core.tools import tool
import requests

@tool
def get_coordinates(location:str)->str:
    """Get latitude and longitude for a location using Open-Meteo Geocoding API"""
    geocoding_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1&language=en&format=json"
    try:
        response = requests.get(geocoding_url)
        response.raise_for_status()
        data = response.json()
        if data.get("results"):
            return data["results"][0]["latitude"], data["results"][0]["longitude"]
        return None
    except Exception as e:
      return None

In [82]:
tools = [get_coordinates]
llm_with_tools = llm.bind_tools(tools)

In [83]:
from langchain_core.messages import HumanMessage, AIMessage

In [84]:
query:str = "What are the coordinates of lahore ?"

In [85]:
messages = [HumanMessage(query)]

In [86]:
ai_msg_tools = llm_with_tools.invoke(query)

In [87]:
messages.append(ai_msg_tools)

In [88]:
from langchain_core.messages import HumanMessage, AIMessage
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
      "get_coordinates":get_coordinates
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)

In [89]:
messages.append(tool_msg)

In [90]:
response = llm_with_tools.invoke(messages)

In [91]:
print(response.content)

The coordinates of Lahore are 31.558° N, 74.35071° E.
